<a href="https://colab.research.google.com/github/e1y4d/topera/blob/main/Recsys%5BNo%20NLP%20-%20No%20vector%20embedding%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import json
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity
from google.colab import drive


drive.mount('/content/drive')
path = '/content/drive/My Drive/user_response.json'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [140]:
le = LabelEncoder()
data = pd.read_json(path)
newdata = data.dropna()
newerdata = newdata.copy()
global recommendations, k

In [145]:
def recommend_collaborators(user_id, k=5):

  recommendations = {}  # Dictionary to store recommendations

  # Create a similarity matrix excluding TrackLearned
  filtered = newerdata.drop('TrackLearned', axis=1).copy()
  filtered['UserId'] = filtered['UserId'].astype(str)  # Ensure string user IDs
  encoded_data = filtered.apply(le.fit_transform, axis=0)
  similarity_matrix = cosine_similarity(encoded_data)
  similarity_df = pd.DataFrame(similarity_matrix, index=filtered['UserId'], columns=filtered['UserId'])

  # Recommend for each technology (excluding user's learned tracks)
  for technology in newerdata['TrackLearned'].explode().unique():
    filtered_users = newerdata[newerdata['TrackLearned'] == technology]
    filtered_users = filtered_users.drop('TrackLearned', axis=1)
    encoded_filtered = filtered_users.apply(le.fit_transform, axis=0)
    user_similarity = cosine_similarity(encoded_filtered)
    similar_scores = pd.Series(user_similarity[0], index=filtered_users['UserId'])
    similar_users = similar_scores.sort_values(ascending=False)[1:k+1].index.tolist()
    recommendations.setdefault(technology, similar_users)[:k] = similar_users[:k]  # Truncate to top K


  # Return final recommendations (deduplicate)
  for technology, users in recommendations.items():
    recommendations[technology] = list(set(users))  # Remove duplicates
    return recommendations

# Example usage
recommend_collaborators('ad70ff26-d4d2-4e46-a789-305fc4f6b647')

{'Full-stack': ['4d167a98-431b-4cf8-b27e-d72b3f67b38c',
  '5a251675-3522-474b-afe4-69d94e2fd0e3',
  'a7b36bfc-68b9-48a7-b5db-2b43734eb94c',
  '8e7cb4ae-daae-4969-8f05-35af9ff47d0d'],
 'Frontend': ['ad70ff26-d4d2-4e46-a789-305fc4f6b647',
  'eb1cc09b-681d-479f-b720-cbe4a711c489',
  'f63b22fc-2de8-41a1-a45a-c34da9e547a5',
  'b2f9eae1-fdf4-4861-aeb0-053d46e1964c',
  'e17bda85-98e1-4dc5-a7e5-3fd790116fcf'],
 'Data Science': ['923842f4-6243-4394-8927-7e5306ed2d45',
  'be935bae-6b52-4c3b-b7bb-22fe8fbbf4d3',
  '24f8188f-222b-4b4e-8144-4b3b70c956fc'],
 'Backend': ['d72c4a24-5f9c-49d0-b8cb-d993cb47ef8b',
  'febf4b16-4bc9-4fd7-b489-7a95c5c0da12',
  'a2a29f68-34b4-429d-8d47-f010375c9a9d',
  '2e84d2b1-2317-4b9b-a4d1-1ef47770f4dc']}

In [142]:
def refresh_recommendations():
    """
    Performs recommendation refresh for each technology, excluding previously recommended users.
    """
    excluded_users={}
    for technology in newerdata['TrackLearned'].explode().unique():
      filtered_users = newerdata[~newerdata['UserId'].isin(excluded_users)]  # Exclude already recommended users
      filtered_users = filtered_users[filtered_users['TrackLearned'] == technology]
      filtered_users = filtered_users.drop('TrackLearned', axis=1)  # Exclude TrackLearned from recommendation
      encoded_filtered = filtered_users.apply(le.fit_transform, axis=0)
      user_similarity = cosine_similarity(encoded_filtered)
      similar_scores = pd.Series(user_similarity[0], index=filtered_users['UserId'])
      similar_users = similar_scores.sort_values(ascending=False)[1:k+1].index.tolist()
      recommendations.setdefault(technology, []).extend(similar_users[:k])  # Append recommendations
      excluded_users.update(similar_users[:k])  # Add recommended users to excluded
    # Check if any new recommendations were found
    return recommendations != {}

In [146]:
def recommend_peers(user_id, k=5):
  recommendations = {}

  for technology in newerdata['TrackLearned'].explode().unique():
    filtered = pd.DataFrame(newerdata[newerdata["TrackLearned"] == technology])
    filtered['UserId'] = filtered['UserId'].astype(str)
    encoded_data = filtered.apply(le.fit_transform, axis=0)
    similarity_matrix = cosine_similarity(encoded_data)
    similarity_df = pd.DataFrame(similarity_matrix, index=filtered['UserId'], columns=filtered['UserId'])

    if user_id not in similarity_df.index:
      continue

    similar_scores = similarity_df.loc[user_id].sort_values(ascending=False)
    similar_users = similar_scores[1:k+1].index.tolist()

    recommendations.setdefault(technology, []).extend(similar_users)

  return recommendations

# Example
recommend_peers('ad70ff26-d4d2-4e46-a789-305fc4f6b647')

{'Frontend': ['b2f9eae1-fdf4-4861-aeb0-053d46e1964c',
  'eb1cc09b-681d-479f-b720-cbe4a711c489',
  'f63b22fc-2de8-41a1-a45a-c34da9e547a5',
  '5e33aa21-5b2c-45df-a81a-027272f8f722',
  'e17bda85-98e1-4dc5-a7e5-3fd790116fcf']}